# Example Use-Case
Let's look at a use-case - Two friends living in different parts of a city want to hang out at a restaurant. Due to busy schedules, each can only drive at the max of 30 minutes away from their house.  Let's see how we can use different Here Location Services to resolve their dilemma. 

<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have API key. 
                Please see <a href="https://github.com/heremaps/here-location-services-python/blob/master/docs/notebooks/README.md">README.md</a> for more info on API key.
</div>

In [ ]:
import os
os.environ["LS_API_KEY"] = "MY-API-KEY" # replace your API key here.

In [7]:
import os
from datetime import datetime

from here_location_services import LS
from here_map_widget import Map, Marker, GeoJSON, Group
from here_location_services.config.isoline_routing_config import RANGE_TYPE, ISOLINE_ROUTING_TRANSPORT_MODE


LS_API_KEY = os.environ.get("LS_API_KEY")
ls = LS(api_key=LS_API_KEY)

result1 = ls.calculate_isoline(
    origin=[19.1503, 72.8530],
    range="1800",
    departure_time=datetime.now(),
    range_type=RANGE_TYPE.time,
    transport_mode=ISOLINE_ROUTING_TRANSPORT_MODE.car,
)

data1 = result1.to_geojson()
first_center = Marker(lat=19.1503, lng=72.8530)
geo_layer1 = GeoJSON(data=data1)
result2 = ls.calculate_isoline(
    origin=[19.1621, 73.0008],
    range="1800",
    departure_time=datetime.now(),
    range_type=RANGE_TYPE.time,
    transport_mode=ISOLINE_ROUTING_TRANSPORT_MODE.car,
)
data2 = result2.to_geojson()

geo_layer2 = GeoJSON(data=data2)

second_center = Marker(lat=19.1621, lng=73.0008)

m = Map(
    api_key=LS_API_KEY,
    center=[19.1621, 73.0008],
    zoom=9,
)

In [8]:
m.add_layer(geo_layer1)
m.add_object(first_center)
m

Map(api_key='TKURWbpJHcbzmWMrmTZwGAdSTLrX8WY1oddj4S4U-EM', basemap=None, center=[19.1621, 73.0008], layers=(Ge…

In [9]:
m.add_layer(geo_layer2)
m.add_object(second_center)

In [14]:
from shapely.geometry import Polygon
import shapely.wkt

l1 = data1["features"][0]["geometry"]["coordinates"][0]
l2 = data2["features"][0]["geometry"]["coordinates"][0]
p1 = Polygon(l1)
p2 = Polygon(l2)
p3 = shapely.wkt.loads(str(p1.intersection(p2)))
centroid = p3.centroid
center_marker = Marker(lng=centroid.x, lat=centroid.y)
m.add_object(center_marker)
m

ModuleNotFoundError: No module named 'shapely'

In [11]:
search = ls.discover(center=[centroid.x, centroid.y], limit=10, query='restaurant', country_codes=["IND"])
search.items

NameError: name 'centroid' is not defined

In [ ]:
group = Group(volatility=True)

restaurants =[]
for restaurant in search.items:
    restaurants.append(
        Marker(
            lat=restaurant["position"]["lat"],
            lng=restaurant["position"]["lng"],
            data=restaurant["address"]["label"],
        )
    )
group.add_objects(restaurants)
m.add_object(group)